# Loading Unified Model output

For convenience, `aeolus` provides a way of keeping loaded and processed data within one object along with extra metadata.
The object is called `Run` (as in "simulation run"). 
The code below provides an example of basic usage of `Run`.

In [1]:
from pathlib import Path

import iris

from aeolus.core import Run

Note that you can use either a single filename or a list of filenames, each of which is either a `str` or (recommended) `pathlib.Path` object.

In [2]:
sample_file = Path.cwd() / "sample_data" / "sample_t1e_2d_mean.pp"

While instantiating `Run`, it is possible to add a short name, a long description of the experiment; and to specify a planet configuration with relevant constants (see "Physical constants" example for more info).

In [3]:
my_run = Run(
    files=sample_file,
    name="t1e_example",
    description="This is some sample data from a UM simulation of tidally-locked Trappist-1e planet.",
    planet="trap1e",  # this reads constants from a JSON file
)

In [4]:
my_run

Constants that have been used in the model:

In [5]:
my_run.const

Trap1eConstants(earth_day [s], stefan_boltzmann [W m-2 K-4], water_heat_vaporization [m2 s-2], water_molecular_weight [kg mol-1], molar_gas_constant [J K-1 mol-1], boltzmann [m^2 kg s^-2 K^-1], avogadro [mol-1], gravity [m s-2], radius [m], day [s], solar_constant [W m-2], reference_surface_pressure [Pa], semi_major_axis [au], eccentricity [1], obliquity [degree], dry_air_spec_heat_press [m2 s-2 K-1], dry_air_molecular_weight [kg mol-1], condensible_density [kg m-3], condensible_heat_vaporization [m2 s-2])

The loaded data are stored as a `CubeList` under `raw` attribute.

In [6]:
print(my_run.raw)

0: convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
1: convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
2: high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
3: low_type_cloud_area_fraction / (1)  (latitude: 90; longitude: 144)
4: medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
5: stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
6: stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)


## Minimal processing of loaded data

The next step might involve some clean-up and post-processing of raw data.

This is done by creating a function that takes `iris.cube.CubeList` as its 1st argument and returns another `iris.cube.CubeList` as output.
The function is then passed to `Run.proc_data()` and its output stored as `Run.proc` attribute.

For example, the function `roll_cube_pm180()` imported below takes a `Cube` and rolls its longitudes from 0...360 to -180...180.

In [7]:
from aeolus.coord import ensure_bounds, roll_cube_pm180

It can then be applied to all `raw` cubes.

In [8]:
def _prepare_cubes(cubelist):
    """Post-process data for easier analysis."""
    # Roll cubes
    r_cubes = iris.cube.CubeList()
    for cube in cubelist:
        r_c = roll_cube_pm180(cube)
        ensure_bounds(r_c)  # also, ensure that longitudes and latitudes have bounds
        r_cubes.append(r_c)

    return r_cubes

In [9]:
my_run.proc_data(_prepare_cubes)

In [10]:
print(my_run.proc)

0: convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
1: convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
2: high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
3: low_type_cloud_area_fraction / (1)  (latitude: 90; longitude: 144)
4: medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
5: stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
6: stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)


Check that it did what expected.

In [11]:
print(my_run.raw.extract_cube("convective_rainfall_flux").coord("longitude")[:10])

DimCoord(array([ 1.25,  3.75,  6.25,  8.75, 11.25, 13.75, 16.25, 18.75, 21.25,
       23.75], dtype=float32), standard_name='longitude', units=Unit('degrees'), coord_system=GeogCS(6371229.0))


In [12]:
print(my_run.proc.extract_cube("convective_rainfall_flux").coord("longitude")[:10])

DimCoord(array([-178.75, -176.25, -173.75, -171.25, -168.75, -166.25, -163.75,
       -161.25, -158.75, -156.25]), bounds=array([[-180. , -177.5],
       [-177.5, -175. ],
       [-175. , -172.5],
       [-172.5, -170. ],
       [-170. , -167.5],
       [-167.5, -165. ],
       [-165. , -162.5],
       [-162.5, -160. ],
       [-160. , -157.5],
       [-157.5, -155. ]]), standard_name='longitude', units=Unit('degrees'), coord_system=GeogCS(5804071.0))


Note that the longitude coordinate is not only shifted by 180 degrees, but has automatically calculated `bounds`.

In addition, note that `coord_system` in `proc` cubes is different, because it used `Run.const` attribute to redefine the planet radius correctly.
By default the loaded raw data has Earth radius in its `coord_system`, so certain calculations (e.g. grid cell areas) might be incorrect.